In [10]:
import cv2
import time
import numpy as np
from keras.models import load_model


def getPredictedClass(model, vimg):
    image = np.zeros((200, 200, 3)) 
    image[:,:,0] = vimg[:,:,0]/255
    image[:,:,1] = vimg[:,:,1]/255
    image[:,:,2] = vimg[:,:,2]/255

    image = cv2.resize(image, (200, 200))
    image = image.reshape(1, 200, 200, 3)

    prediction = model.predict_on_batch(image)
    predicted_class = np.argmax(prediction)
    if predicted_class == 26:
        return "nothing"
    elif predicted_class == 27:
        return "delete"
    elif predicted_class == 28:
        return "space"
    return str(chr(predicted_class + ord('A')))

if __name__ == "__main__":
    camera = cv2.VideoCapture(0)
    fps = int(camera.get(cv2.CAP_PROP_FPS))
    top, right, bottom, left = 30, 50, 230, 250
    model = load_model("del3.0/model-00003-0.16938-0.95240-0.46589-0.89400.h5")
    k = 0
    predictedClass = "Loading .."
    while (True):
        (grabbed, frame) = camera.read()
        if grabbed:
            frame = cv2.resize(frame, (700,700))
            # flip the frame so that it is not the mirror view
            frame = cv2.flip(frame, 1)
            clone = frame.copy()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            (height, width) = frame.shape[:2]
            roi = frame[top:bottom, right:left]
            if k % (fps / 3) == 0:
                predictedClass = getPredictedClass(model, roi)

            cv2.putText(clone, str(predictedClass), (490, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

            # display the frame
            cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF
        k += 1
        # if the user pressed "q", exit
        if keypress == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()
